In [1]:
import pandas as pd 
import numpy as np 
import math
import matplotlib.pyplot as plt
import plotly.plotly as py
import plotly.tools as tls
from sklearn import metrics
from sklearn.model_selection import train_test_split
from tqdm import tqdm

import os
import pickle

In [2]:
pos_embedding = pickle.load(open('pos_embedding','rb'))
len(pos_embedding)

6291

In [3]:

pos_embedding[1].shape
type(pos_embedding[0])

numpy.ndarray

In [4]:
neg_embedding = pickle.load(open('neg_embedding','rb'))

In [5]:
len(neg_embedding)

187232

In [6]:
#sub smaple negative samples
neg_embedding=neg_embedding[:6291]
len(neg_embedding)



6291

In [7]:
#concatenating
all_embeddings=list(pos_embedding+neg_embedding)
len(all_embeddings)

12582

In [8]:
#creating two arrays with 1s and 0s
pos_target=np.ones(6291)
pos_target



array([1., 1., 1., ..., 1., 1., 1.])

In [9]:
neg_target=np.zeros(6291)
neg_target

array([0., 0., 0., ..., 0., 0., 0.])

In [10]:
all_target= np.append(pos_target,neg_target,axis=None)
all_target

array([1., 1., 1., ..., 0., 0., 0.])

In [11]:
all_target[:5]

array([1., 1., 1., 1., 1.])

In [12]:
type(all_embeddings)

list

In [13]:
all_embeddings[0].shape

(361, 6)

In [14]:
embedding_size = 361
question_size = 104
def get_padded_embeddings(temp_df):
    #print("tdf", temp_df.shape)
    temp_df=temp_df.reshape(-1,361)
    zero_embeddings = np.zeros(embedding_size)
    truncated_df = temp_df[:question_size]
    #print(truncated_df.shape)
    #print(len(truncated_df),len(truncated_df[0]))
    #print(type([zero_embeddings]*(question_size-len(truncated_df))))
    #print(np.array([zero_embeddings]*(question_size - len(truncated_df))).shape)
    if(len(truncated_df)!=104):
        truncated_df = np.concatenate((truncated_df, np.array([zero_embeddings]*(question_size- len(truncated_df)))))
        #print(truncated_df.shape)
    return truncated_df


In [15]:
emb_vect_temp=np.array([get_padded_embeddings(line) for line in all_embeddings])

In [16]:
emb_vect_temp.shape

(12582, 104, 361)

In [17]:
type(emb_vect_temp)

numpy.ndarray

In [18]:
all_embeddings_train, all_embeddings_test,y_train,y_test = train_test_split(emb_vect_temp, all_target, test_size = 0.30, shuffle=True, random_state = 42)


In [22]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Bidirectional,Flatten
from sklearn.metrics import f1_score
from keras.utils import to_categorical


Using TensorFlow backend.


In [23]:
model = Sequential()
model.add(Bidirectional(LSTM(64, return_sequences=True, input_shape=(104,361))))
model.add(Bidirectional(LSTM(64)))
model.add(Dense(1, activation="sigmoid"))

In [24]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.


In [ ]:
history=model.fit(all_embeddings_train, y_train, batch_size=None,nb_epoch=1,steps_per_epoch=1000, verbose=1)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning:

The `nb_epoch` argument in `fit` has been renamed `epochs`.



Instructions for updating:
Use tf.cast instead.
Epoch 1/1
  94/1000 [=>............................] - ETA: 14:24:13 - loss: 0.5606 - acc: 0.6968

In [ ]:
with open('HistoryDict', 'wb') as file_pi:
        pickle.dump(history.history, file_pi)


In [ ]:
with open('Model', 'wb') as file_mo:
        pickle.dump(model, file_mo)

In [19]:
a=all_embeddings_train[2].reshape(-1,361).shape
a

(104, 361)

In [20]:
a=get_padded_embeddings(all_embeddings_train[2])

In [184]:
#all_embeddings_train[1].reshape(-1,361)
#all_embeddings_train[1].shape

In [393]:
def batch_gen(n_batches,y,all_embeddings):
    #print("bg")
    while True: 
        for i in range(n_batches):
            #print("for loop")
            embeddings_list = all_embeddings[i*batch_size:(i+1)*batch_size] 
            #print("el",embeddings_list.shape,all_embeddings.shape,embeddings_list[0].shape)
            concat_embed_ques = np.array([get_padded_embeddings(ques) for ques in all_embeddings])
            #print(i,concat_embed_ques.shape)
            yield concat_embed_ques, np.array(y[i*batch_size:(i+1)*batch_size])

In [394]:
batch_size=32
n_batches = math.ceil(len(y_train)/batch_size)
bg=batch_gen(n_batches,y_train,all_embeddings_train)

In [392]:
n_batches

276

In [395]:
from keras.models import Sequential
from keras.layers import CuDNNLSTM, Dense, Bidirectional
from sklearn.metrics import f1_score
model = Sequential()
model.add(Bidirectional(CuDNNLSTM(64, return_sequences=True),
                        input_shape=(104, 361)))
model.add(Bidirectional(CuDNNLSTM(64)))
model.add(Dense(1, activation="sigmoid"))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.fit_generator(bg, epochs=2,
                    steps_per_epoch=1000,
                    verbose=True)


Epoch 1/2


AttributeError: 'list' object has no attribute 'shape'

In [330]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_51 (Bidirectio (None, 104, 128)          218624    
_________________________________________________________________
bidirectional_52 (Bidirectio (None, 128)               99328     
_________________________________________________________________
dense_34 (Dense)             (None, 1)                 129       
Total params: 318,081
Trainable params: 318,081
Non-trainable params: 0
_________________________________________________________________


In [336]:
test_gen=batch_gen(n_batches,y_test,all_embeddings_test)
scores=model.evaluate_generator(test_gen,steps=100,verbose=1)
print("Accuracy", scores[1])
pred_val=model.predict_generator(test_gen, steps=100,verbose=1)
print("Predicted val", pred_val[1])

ValueError: Error when checking input: expected bidirectional_53_input to have 3 dimensions, but got array with shape (3775, 1)

In [346]:
emb_vect[0].shape

(104, 361)

In [348]:
x=emb_vect
x=x.reshape(x,104,361)
x.shape

TypeError: only integer scalar arrays can be converted to a scalar index

In [27]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Bidirectional,Flatten
from sklearn.metrics import f1_score
from keras.utils import to_categorical

In [29]:
model = Sequential()
model.add(Bidirectional(LSTM(64, return_sequences=True, input_shape=(104,361))))
model.add(Bidirectional(LSTM(64)))
model.add(Dense(1, activation="sigmoid"))

In [30]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
model.fit(all_embeddings_train, y_train, batch_size=None,nb_epoch=20,steps_per_epoch=1000, verbose=1)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning:

The `nb_epoch` argument in `fit` has been renamed `epochs`.



Epoch 1/20
1000/1000 [==============================] - 55711s 56s/step - loss: 0.3053 - acc: 0.8508
Epoch 2/20
 266/1000 [======>.......................] - ETA: 10:38:01 - loss: 0.2350 - acc: 0.8879

In [25]:
model.summary()         

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 104, 64)           109056    
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                33024     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 142,145
Trainable params: 142,145
Non-trainable params: 0
_________________________________________________________________
